# Equal Experience in Recommender Systems on ML-1m and Synthetic biased data

## Executive summary

| | |
| --- | --- |
| Problem | Biased data due to inherent stereotypes of particular groups (e.g., male students’ average rating on mathematics is often higher than that on humanities, and vice versa for females) may yield a limited scope of suggested items to a certain group of users. |
| Solution | The novel fairness notion, coined ***Equal Experience***, tries to capture the degree of the equal experience of item recommendations across distinct groups. Specifically, the prediction $\hat{Y}$ should be independent of the 1) user group $Z_{user}$, and 2) item group $Z_{item}$. Formally, $I(\hat{Y};Z_{user},Z_{item}) = I(\hat{Y};Z_{item}) + I(\hat{Y};Z_{user}|Z_{item}) = I(\hat{Y};Z_{user}) + I(\hat{Y};Z_{item}|Z_{user})$ |
| Dataset | ML-1m, LastFM-360k, Synthetic. |
| Preprocessing | For ML-1m, we divide user and item groups based on gender and genre, respectively. Action, crime, film-noir, war are selected as male-preferred genre, whereas children, fantasy, musical, romance are selected as female-preferred genre. We can select male-preferred and female-preferred genres in a variety of ways based on ratings and observations. In case of LastFM-360k, the associated task is to predict whether the user likes the artist or not. The data for play counts is converted to binary rating. We divide user and item groups based on gender and genre, respectively. We also randomly select 5000 male and 5000 female users. Among 10 genres, we choose hip-hop and musical for male and female preferred genres, respectively. The final rating matrix of 10,000 users and 5,706 artists is 0.55% full. We randomly split the real datasets into 90% train set and 10% test set. In case of MovieLens data, the rating is five-star based, so we set the threshold $\tau$ = 3, on the other hand, for LastFM and for synthetic dataset, we set $\tau$ = 0 as $M_{ij} \in \{+1, −1\}$. |
| Metrics | RMSE, DEE, VAL, UGF, CVS |
| Models | MF class models, AE class models |
| Cluster | Python 3.6+, PyTorch |
| Tags | `Fairness`, `MatrixFactorization`, `AutoEncoder`,  `ExposureBias`, `PopulationBias` |
| Credits | cjw2525 |

## Process flow

![](https://github.com/RecoHut-Stanzas/S035564/raw/main/images/process_flow.svg)

## Setup

In [ ]:
!pip install livelossplot

In [ ]:
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import math
import collections
import itertools

import os
import sys

import torch
import torch.nn as nn
from torch import optim
from torch.nn import utils
import torch.nn.functional as F

import matplotlib.pyplot as plt
from time import sleep
from livelossplot import PlotLosses  

%matplotlib inline

## Data

### ML-1m

Download

In [ ]:
!wget -q --show-progress http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

ml-1m.zip           100%[===================>]   5.64M  7.06MB/s    in 0.8s    
Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


Preprocessing

In [ ]:
def data_loader_movielens():
    path = './ml-1m/'
    num_users, num_items = 6040, 3952
      
    data = load_data(path, num_users, num_items, train_ratio=.9)
    user, _ = load_users(path)
    genre = load_items(path, option='single')
    item = {}
    item['M'] = genre['War']+genre['Crime']+genre['Film-Noir']+genre['Sci-Fi']
    item['F'] = genre['Children\'s']+genre['Fantasy']+genre['Musical']+genre['Romance']
    
    return data, user, item

In [ ]:
def load_users(path):
    f = open(path + "users.dat")
    lines = f.readlines()

    gender, age = {}, {} # generate dictionaries
    gender_index, age_index = ['M', 'F'], [1, 18, 25, 35, 45, 50, 56]

    for i in gender_index:
        gender[i] = []
    for i in age_index:
        age[i] = []  
    for line in lines:
        user, g, a, *args = line.split("::")
        gender[g].append(int(user) - 1)
        age[int(a)].append(int(user) - 1) 

    return gender, age

In [ ]:
def load_items(path, option='multiple_genre'):
    f = open(path + "movies.dat", encoding = "ISO-8859-1")
    lines = f.readlines()

    genre={}
    genre_index = ['Action', 'Adventure', 'Animation', 'Children\'s', 
                   'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                   'Film-Noir', 'Horror', 'Musical', 'Mystery', 
                   'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

    for idx in genre_index:
        genre[idx] = []

    for line in lines:
        item, _, tags = line.split("::")
        tags = tags.split('|')
        tags[-1] = tags[-1][:-1]
        if option=='multiple_genre':
            for tag in tags:
                genre[tag].append(int(item) - 1)
        else:
            tag = tags[0]
            genre[tag].append(int(item)-1)
    return genre

In [ ]:
def load_data(path, num_users, num_items, train_ratio):
    '''
    Read data by lines and produce train/test data matrices.
    '''

    f = open(path + "ratings.dat")
    lines = f.readlines()
    random.shuffle(lines)  

    num_ratings = len(lines)

    X_train = np.zeros((num_users, num_items))
    X_test = np.zeros((num_users, num_items))

    for i, line in enumerate(lines):
        user, item, rating, _ = line.split("::")
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        if i < int(num_ratings * train_ratio):
            X_train[user_idx, item_idx] = float(rating)
        else:
            X_test[user_idx, item_idx] = float(rating)

    return (X_train, X_test)

### Synthetic dataset

In [ ]:
def data_loader_synthetic(p=.2, q=.2, r=.2, s=.2, rank=10, seed=42):
    '''
    ground truth matrix Y
    '''
    num_users, num_items = 600, 400
    n1, n2 = num_users // 2, num_items // 2
    np.random.seed(42)
    Y_1 = np.where(np.random.random((rank, n2)) < p, 1, -1)
    np.random.seed(42)
    Y_2 = np.where(np.random.random((rank, n2)) < q, 1, -1)
    Y_rank = np.concatenate((Y_1, Y_2), axis = 1)

    Y_m = Y_rank.copy()

    for i in range(num_users // (rank * 2) -1):
        Y_m = np.concatenate((Y_m, Y_rank))
    np.random.seed(43)
    Y_1 = np.where(np.random.random((rank, n2)) < q, 1, -1)
    np.random.seed(43)
    Y_2 = np.where(np.random.random((rank, n2)) < p, 1, -1)
    Y_rank = np.concatenate((Y_1, Y_2), axis = 1)

    Y_f = Y_rank.copy()

    for i in range(num_users // (rank * 2) -1):
        Y_f = np.concatenate((Y_f, Y_rank))
    
    np.random.shuffle(Y_m)
    np.random.shuffle(Y_f)
    Y = np.concatenate((Y_m, Y_f))
    
    I_obs_mm = np.where(np.random.random((n1, n2)) < r, 1, 0)
    I_obs_mf = np.where(np.random.random((n1, n2)) < s, 1, 0)
    I_obs_fm = np.where(np.random.random((n1, n2)) < s, 1, 0)
    I_obs_ff = np.where(np.random.random((n1, n2)) < r, 1, 0)

    I_obs_m = np.concatenate((I_obs_mm, I_obs_mf), axis = 1)
    I_obs_f = np.concatenate((I_obs_fm, I_obs_ff), axis = 1)

    I_obs = np.concatenate((I_obs_m, I_obs_f))
    
    Y_obs = Y * I_obs
    
    Y_train, Y_test = Y_obs, (Y-Y_obs)
#     Y_train, Y_test = np.zeros((num_users, num_items)), np.zeros((num_users, num_items))

#     for i in tqdm(range(num_users)):
#         for j in range(num_items):
#             if Y_obs[i, j] != 0:
#                 k = np.random.random()
#                 if k > 0.9:
#                     Y_test[i, j] = Y_obs[i, j]
#                 else:
#                     Y_train[i, j] = Y_obs[i, j]
                    
                    
    user, item = {}, {}
    user['M'] = [x for x in range(n1)]
    user['F'] = [x for x in range(n1, n1*2)]
    item['M'] = [x for x in range(n2)]
    item['F'] = [x for x in range(n2, n2*2)]
                    
    return (Y_train, Y_test), user, item

## Metrics

In [ ]:
def metrics(model, data_tuple, device, model_type='AE', tau=3):
    
    data, gender, item = data_tuple
    measures = {}
    
    with torch.no_grad(): 
        model.eval()
        Y_train, Y_test = data[0], data[1]
        if model_type=='PQ':
            identity = torch.from_numpy(np.eye(Y_train.shape[0])).float().to(device)
            pred = model(identity).cpu().detach().numpy()
        else:
            pred = model(torch.tensor(Y_train).float().to(device)).cpu().detach().numpy()   
        # 1. rmse
        test_rmse = np.sqrt(np.mean((Y_test[Y_test != 0] - pred[Y_test != 0]) ** 2))
        # Y_tilde 
        pred_hat = np.where(pred > tau, 1, 0)
        
        # 2. DEE
        DEE = 0
        for g in ['M', 'F']:
            for i in ['M', 'F']:
                DEE += np.abs(np.mean(pred_hat)-np.mean(pred_hat[gender[g]][:, item[i]]))
        # 3. value_fairness
        VAL = VAL_measure(pred, data, gender, device)
        # 4. DP_user
        UGF = 0
        for g in ['M', 'F']:
            UGF += np.abs(np.mean(pred_hat)-np.mean(pred_hat[gender[g]]))
        # 4. DP_item
        CVS = 0
        for i in ['M', 'F']:
            CVS += np.abs(np.mean(pred_hat)-np.mean(pred_hat[:, item[i]]))
        measures['RMSE'] = test_rmse
        measures['DEE'] = DEE
        measures['VAL'] = VAL 
        measures['UGF'] = UGF
        measures['CVS'] = CVS
    return measures

In [ ]:
def VAL_measure(pred, data, gender, device):
    train_data = data[0]
    mask = np.where(train_data!=0, 1, 0)

    y_m = train_data[gender['M']]
    y_f = train_data[gender['F']]
    y_hat_m = pred[gender['M']]
    y_hat_f = pred[gender['F']]

    #average ratings
    d_m = np.abs(np.sum(y_m, axis=0)/(np.sum(mask[gender['M']], axis=0)+1e-8)-np.sum(y_hat_m, axis=0)/len(gender['M']))
    d_f = np.abs(np.sum(y_f, axis=0)/(np.sum(mask[gender['F']], axis=0)+1e-8)-np.sum(y_hat_f, axis=0)/len(gender['F']))

    v_fairness = np.mean(np.abs(d_m-d_f))
    return v_fairness

## Regularizers

In [ ]:
def normal_pdf(x):
    import math
    return torch.exp(-0.5 * x**2) / math.sqrt(2 * math.pi)

def normal_cdf(y, h=0.01, tau=0.5):
    # Approximation of Q-function given by López-Benítez & Casadevall (2011)
    # based on a second-order exponential function & Q(x) = 1 - Q(-x):
    Q_fn = lambda x: torch.exp(-0.4920*x**2 - 0.2887*x - 1.1893)
    m = y.shape[0]*y.shape[1]
    y_prime = (tau - y) / h
    sum_ = torch.sum(Q_fn(y_prime[y_prime > 0])) \
           + torch.sum(1 - Q_fn(torch.abs(y_prime[y_prime < 0]))) \
           + 0.5 * len(y_prime[y_prime == 0])
    return sum_ / m

def Huber_loss(x, delta):
    if abs(x) < delta:
        return (x ** 2) / 2
    return delta * (x.abs() - delta / 2)

def Huber_loss_derivative(x, delta):
    if x > delta:
        return delta/2
    elif x < -delta:
        return -delta/2
    return x

In [ ]:
class FairnessLoss():
    def __init__(self, h, tau, delta, device, data_tuple, type_='EqualExp'):
        self.h = h
        self.tau = tau
        self.delta = delta
        self.device = device
        self.type_ = type_
        self.data_tuple = data_tuple

    def DEE(self, y_hat, gender, item):
        backward_loss = 0
        logging_loss_ = 0 
        
        for gender_key in ['M','F']:
            for item_key in ['M', 'F']:
                gender_idx = gender[gender_key] 
                item_idx = item[item_key]
                m_gi = len(gender_idx)*len(item_idx)
                y_hat_gender_item = y_hat[gender_idx][:, item_idx]

                Prob_diff_Z = normal_cdf(y_hat.detach(), self.h, self.tau)-normal_cdf(y_hat_gender_item.detach(), self.h, self.tau)
                
                _dummy = Huber_loss_derivative(Prob_diff_Z, self.delta)
                _dummy *= \
                    torch.dot(
                        normal_pdf((self.tau - y_hat.detach()) / self.h).reshape(-1), 
                        y_hat.reshape(-1)
                    ) / (self.h * y_hat.shape[0]*y_hat.shape[1]) -\
                    torch.dot(
                        normal_pdf((self.tau - y_hat_gender_item.detach()) / self.h).reshape(-1), 
                        y_hat_gender_item.reshape(-1)
                    ) / (self.h * m_gi)
                backward_loss += _dummy
        return backward_loss
        
    def VAL(self, y_hat, gender, item):
        device = self.device
        
        backward_loss = 0
        
        data = self.data_tuple[0]
        train_data = data[0]
        mask = np.where(train_data!=0, 1, 0)

        train_data = torch.from_numpy(train_data).to(device)
        mask = torch.from_numpy(mask).to(device)

        y_m = train_data[gender['M']]
        y_f = train_data[gender['F']]
        y_hat_m = y_hat[gender['M']]
        y_hat_f = y_hat[gender['F']]

        #average ratings
        d_m = torch.abs(torch.sum(y_m, axis=0)/(torch.sum(mask[gender['M']], axis=0)+1e-8)
        -torch.sum(y_hat_m, axis=0)/len(gender['M']))

        d_f = torch.abs(torch.sum(y_f, axis=0)/(torch.sum(mask[gender['F']], axis=0)+1e-8)
        -torch.sum(y_hat_f, axis=0)/len(gender['F']))


        backward_loss = torch.mean(torch.abs(d_m-d_f))
        
        return backward_loss
    
    def UGF(self, y_hat, gender, item):
        backward_loss = 0
        
        for key in ['M', 'F']:
            
            gender_idx = gender[key]
            m_i = y_hat.shape[1]*len(gender_idx)
            y_hat_group = y_hat[gender_idx]
            
            Prob_diff_Z = normal_cdf(y_hat.detach(), self.h, self.tau)-normal_cdf(y_hat_group.detach(), self.h, self.tau)

            _dummy = Huber_loss_derivative(Prob_diff_Z, self.delta)
            _dummy *= \
                torch.dot(
                    normal_pdf((self.tau - y_hat.detach()) / self.h).reshape(-1), 
                    y_hat.reshape(-1)
                ) / (self.h * y_hat.shape[0]*y_hat.shape[1]) -\
                torch.dot(
                    normal_pdf((self.tau - y_hat_group.detach()) / self.h).reshape(-1), 
                    y_hat_group.reshape(-1)
                ) / (self.h * m_i)
            backward_loss += _dummy
        return backward_loss
    
    def CVS(self, y_hat, gender, item):
        backward_loss = 0
        
        for key in ['M', 'F']:
            item_idx = item[key]
            m_i = y_hat.shape[0]*len(item_idx)
            y_hat_group = y_hat[:, item_idx]

            Prob_diff_Z = normal_cdf(y_hat.detach(), self.h, self.tau)-normal_cdf(y_hat_group.detach(), self.h, self.tau)

            _dummy = Huber_loss_derivative(Prob_diff_Z, self.delta)
            _dummy *= \
                torch.dot(
                    normal_pdf((self.tau - y_hat.detach()) / self.h).reshape(-1), 
                    y_hat.reshape(-1)
                ) / (self.h * y_hat.shape[0]*y_hat.shape[1]) -\
                torch.dot(
                    normal_pdf((self.tau - y_hat_group.detach()) / self.h).reshape(-1), 
                    y_hat_group.reshape(-1)
                ) / (self.h * m_i)
            backward_loss += _dummy
        return backward_loss
        
    
    def __call__(self, y_hat, gender, item):
        if self.type_ == 'EqualExp':
            return self.DEE(y_hat, gender, item)
        elif self.type_ == 'VAL':
            return self.VAL(y_hat, gender, item)
        elif self.type_ == 'UGF':
            return self.UGF(y_hat, gender, item)
        elif self.type_ == 'CVS':
            return self.CVS(y_hat, gender, item)

## Models

### Matrix factorization

In [ ]:
class PQ(nn.Module):
    def __init__(self, rating, num_users, num_items, rank):
        super(PQ, self).__init__()
        
        self.rating = rating
        
        self.encoder = nn.Sequential(nn.Linear(num_users, rank, bias=False))
        self.decoder = nn.Sequential(nn.Linear(rank, num_items, bias=False))

    def forward(self, x):
        if self.rating == 'binary':
            x = self.decoder(self.encoder(x))
            x = torch.tanh(x)
        elif self.rating == 'five-stars':
            x = self.decoder(self.encoder(x))
            x = torch.clamp(x, 0, 5.0)
        else:
            raise KeyError("unavailable rating scale")
    
        return x

### Autoencoder

In [ ]:
class AE(nn.Module):
    def __init__(self, rating, num_user):
        super(AE, self).__init__()
        
        self.rating = rating
        self.encoder = nn.Sequential(
          nn.Linear(num_user, 512),
          nn.ReLU(),
          nn.Linear(512, 512),
          nn.Dropout(0.7),
          nn.ReLU(),
        )
        self.decoder = nn.Sequential(
          nn.Linear(512, num_user),
        )
        
    def forward(self, x):
        x = torch.transpose(x,0,1)
        if self.rating == 'binary':
            x = self.decoder(self.encoder(x))
            x = torch.tanh(x)
        elif self.rating == 'five-stars':
            x = (x - 1) / 4.0
            x = self.decoder(self.encoder(x))
            x = torch.clamp(x, 0, 1.0)
            x = 4.0 * x + 1
        x = torch.transpose(x,0,1)
        return x

## Trainer

In [ ]:
def mklogs():
    logs = {'train_loss':[], 
            'train_f_loss':[], 
            'RMSE':[],
            'acc':[], 
            'DEE':[], 
            'DP_user':[], 
            'DP_item':[], 
            'v_fairness':[]
           }
    return logs 

In [ ]:
def get_test_logs(logs, log):
    measures=['RMSE',
              'acc', 
              'DEE', 
              'DP_user', 
              'DP_item', 
              'v_fairness']
    for measure in measures:
        logs[measure].append(log[measure])

In [ ]:
def train_PQ(data_tuple, model, optimizer, 
             num_epochs, device, l_value=0., lambda_=0., f_criterion=None, tau=3):

    logs = mklogs()
    data, gender, item = data_tuple
    
    # data_input 
    train_data = torch.from_numpy(data[0]).float().to(device) 
    test_data = data[1]
    
    identity = torch.from_numpy(np.eye(data[0].shape[0])).float().to(device)
    
    x = train_data
    mask = x.clone().detach()
    mask = torch.where(mask != 0, torch.ones(1).to(device), torch.zeros(1).to(device)).float().to(device)
    count = torch.sum(mask).item()
    
    #losses
    criterion = nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        rmse, cost = 0, 0
        model.train()
        W, V = model.encoder[0].weight, model.decoder[0].weight
        W_fro, V_fro = torch.sum(W ** 2), torch.sum(V ** 2)
        
        x_hat = model(identity)
        loss = 0 
        loss += (1-lambda_)*(criterion(x * mask, x_hat * mask)/count + l_value / 2 * ( W_fro + V_fro ))
        if f_criterion!=None: 
            f_loss = f_criterion(x_hat, gender, item)
            logs['train_f_loss'].append(f_loss.item())
            loss += lambda_*f_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        logs['train_loss'].append(loss.item())
        
    return logs

In [ ]:
def train_AE(data_tuple, model, optimizer, 
             num_epochs, device, l_value=0., lambda_=0., f_criterion=None, tau=3):

    logs = mklogs()
    data, gender, item = data_tuple
    
    # data_input 
    train_data = torch.from_numpy(data[0]).float().to(device) 
    test_data = data[1]
    
    x = train_data
    mask = x.clone().detach()
    mask = torch.where(mask != 0, torch.ones(1).to(device), torch.zeros(1).to(device)).float().to(device)
    count = torch.sum(mask).item()
    
    #losses
    criterion = nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        rmse, cost = 0, 0
        model.train()
        W, V = model.encoder[0].weight, model.decoder[0].weight
        W_fro, V_fro = torch.sum(W ** 2), torch.sum(V ** 2)
        
        x_hat = model(x)
        loss = 0 
        loss += (1-lambda_)*(criterion(x * mask, x_hat * mask)/count + l_value / 2 * ( W_fro + V_fro ))
        if f_criterion!=None: 
            f_loss = f_criterion(x_hat, gender, item)
            logs['train_f_loss'].append(f_loss.item())
            loss += lambda_*f_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        logs['train_loss'].append(loss.item())
        
    return logs

## Run

In [ ]:
class Args:
    def __init__(self, dataset='movielens'):
        self.dataset = dataset
        self.p0, self.p1 = 0.4, 0.1
        self.q0, self.q1 = 0.2, 0.2
        self.data_type = 'binary'
        self.model_type='PQ' # Choose model type: 'PQ' or 'AE'
        self.algorithm_type = 'EqualExp' # Choose algorithm: 'unfair', 'EqualExp', 'VAL', 'UGF', 'CVS'
        if self.dataset=='movielens':
            self.data_type = 'five-stars'
            self.data_tuple = (data_loader_movielens())
        # elif self.dataset=='lastfm':
        #     self.data_tuple = (data_loader_lastfm())
        elif self.dataset=='synthetic':
            self.data_tuple = (data_loader_synthetic(self.p0, self.p1, self.q0, self.q1)) # return ((train_data, test_data), user attribute, item attribute)
        self.learning_rate = 1e-3
        self.l_value = 0
        self.num_epochs = 1000
        self.lambda_ = 0.99
        self.tau=0
        self.n, self.m = self.data_tuple[0][0].shape[0], self.data_tuple[0][0].shape[1]
        self.r = 20
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        if self.algorithm_type == 'unfair':
            self.f_criterion = None 
        else: 
            self.f_criterion = FairnessLoss(h=0.01, tau=self.tau, delta=0.01,
                                            device=self.device, 
                                            data_tuple=self.data_tuple, 
                                            type_=self.algorithm_type)

### Experiment on ML-1m

In [ ]:
args = Args(dataset='movielens')

results = []

model_type = ['PQ','AE']
algorithm_type = ['unfair', 'EqualExp', 'VAL', 'UGF', 'CVS']

for m in model_type:
    for algo in algorithm_type:
        try:
            args.model_type = m
            args.algorithm_type = algo
            # train the model
            if args.model_type == 'PQ':
                model = PQ(args.data_type, args.n, args.m, 20).to(args.device)
                optimizer = optim.Adam(model.parameters(), lr = args.learning_rate)
                logs = train_PQ(args.data_tuple, model, optimizer, args.num_epochs,
                                args.device, l_value=args.l_value, lambda_=args.lambda_,
                                f_criterion=args.f_criterion, tau=args.tau)
            elif args.model_type == 'AE':
                model = AE(args.data_type, args.n).to(args.device)
                optimizer = optim.Adam(model.parameters(), lr = args.learning_rate)
                logs = train_AE(args.data_tuple, model, optimizer, args.num_epochs,
                                args.device, l_value=args.l_value, lambda_=args.lambda_,
                                f_criterion=args.f_criterion, tau=args.tau)
                
            result = metrics(model, args.data_tuple, args.device, args.model_type, args.tau)
            print(result)
            results.append((m, algo, result))
        except:
            pass

{'RMSE': 0.9027500699012365, 'DEE': 0.0012048745340699218, 'VAL': 0.31501372873975914, 'UGF': 0.0002971521526640153, 'CVS': 0.0006153119059790768}
{'RMSE': 0.9015563787906975, 'DEE': 0.001095268864150678, 'VAL': 0.30451721083300864, 'UGF': 0.0001496180935723901, 'CVS': 0.0005326444826091459}
{'RMSE': 0.9133768224920821, 'DEE': 0.0009910946781319652, 'VAL': 0.307643166842646, 'UGF': 6.867560418755136e-05, 'CVS': 0.00048708791966778353}
{'RMSE': 0.9130855504041286, 'DEE': 0.001214569441231883, 'VAL': 0.30696451034405997, 'UGF': 2.6442839102136517e-05, 'CVS': 0.0005941362836467956}
{'RMSE': 0.9142809402166817, 'DEE': 0.0009662139592129249, 'VAL': 0.3151116659819781, 'UGF': 1.7062581478044514e-05, 'CVS': 0.0004522546364315039}
{'RMSE': 0.8623466023698251, 'DEE': 0.0, 'VAL': 0.3562827773780338, 'UGF': 0.0, 'CVS': 0.0}
{'RMSE': 0.8626359054188043, 'DEE': 0.0, 'VAL': 0.3361190263181524, 'UGF': 0.0, 'CVS': 0.0}
{'RMSE': 0.8609437988471073, 'DEE': 0.0, 'VAL': 0.3366856844320138, 'UGF': 0.0, 'CV

In [ ]:
df_ml = pd.DataFrame.from_records(results)
df_ml.columns = ['Model','Algorithm','Metrics']
df_ml = pd.concat([df_ml.drop('Metrics', axis=1), pd.DataFrame(df_ml['Metrics'].tolist())], axis=1)
df_ml

,Model,Algorithm,RMSE,DEE,VAL,UGF,CVS
0,PQ,unfair,0.902750,0.001205,0.315014,0.000297,0.000615
1,PQ,EqualExp,0.901556,0.001095,0.304517,0.000150,0.000533
2,PQ,VAL,0.913377,0.000991,0.307643,0.000069,0.000487
3,PQ,UGF,0.913086,0.001215,0.306965,0.000026,0.000594
4,PQ,CVS,0.914281,0.000966,0.315112,0.000017,0.000452
5,AE,unfair,0.862347,0.000000,0.356283,0.000000,0.000000
6,AE,EqualExp,0.862636,0.000000,0.336119,0.000000,0.000000
7,AE,VAL,0.860944,0.000000,0.336686,0.000000,0.000000
8,AE,UGF,0.862301,0.000000,0.346637,0.000000,0.000000
9,AE,CVS,0.864432,0.000000,0.336751,0.000000,0.000000


In [ ]:
args = Args(dataset='synthetic')

results = []

model_type = ['PQ','AE']
algorithm_type = ['unfair', 'EqualExp', 'VAL', 'UGF', 'CVS']

for m in model_type:
    for algo in algorithm_type:
        try:
            args.model_type = m
            args.algorithm_type = algo
            # train the model
            if args.model_type == 'PQ':
                model = PQ(args.data_type, args.n, args.m, 20).to(args.device)
                optimizer = optim.Adam(model.parameters(), lr = args.learning_rate)
                logs = train_PQ(args.data_tuple, model, optimizer, args.num_epochs,
                                args.device, l_value=args.l_value, lambda_=args.lambda_,
                                f_criterion=args.f_criterion, tau=args.tau)
            elif args.model_type == 'AE':
                model = AE(args.data_type, args.n).to(args.device)
                optimizer = optim.Adam(model.parameters(), lr = args.learning_rate)
                logs = train_AE(args.data_tuple, model, optimizer, args.num_epochs,
                                args.device, l_value=args.l_value, lambda_=args.lambda_,
                                f_criterion=args.f_criterion, tau=args.tau)
                
            result = metrics(model, args.data_tuple, args.device, args.model_type, args.tau)
            print(result)
            results.append((m, algo, result))
        except:
            pass

{'RMSE': 0.6843796436520339, 'DEE': 0.01281666666666667, 'VAL': 0.1854790271175086, 'UGF': 0.00025833333333333264, 'CVS': 0.0001916666666666733}
{'RMSE': 0.6746158616275792, 'DEE': 0.012016666666666648, 'VAL': 0.2008466635558418, 'UGF': 0.00044166666666667354, 'CVS': 0.0001916666666666733}
{'RMSE': 0.6793149919984537, 'DEE': 0.012683333333333324, 'VAL': 0.18660142754964093, 'UGF': 0.00019166666666664556, 'CVS': 0.0004250000000000087}
{'RMSE': 0.6832201142217881, 'DEE': 0.012466666666666654, 'VAL': 0.17845580133450958, 'UGF': 0.00016666666666667607, 'CVS': 0.0005499999999999949}
{'RMSE': 0.6938836450573082, 'DEE': 0.012716666666666682, 'VAL': 0.17499607298075928, 'UGF': 4.166666666668983e-05, 'CVS': 0.00032499999999999196}
{'RMSE': 0.7856401140827461, 'DEE': 0.00470000000000001, 'VAL': 0.23203744433583628, 'UGF': 0.0016333333333333477, 'CVS': 0.001366666666666655}
{'RMSE': 0.7835545178404618, 'DEE': 0.0076833333333333476, 'VAL': 0.22803320192722268, 'UGF': 0.001758333333333334, 'CVS': 0

In [ ]:
df_synthetic = pd.DataFrame.from_records(results)
df_synthetic.columns = ['Model','Algorithm','Metrics']
df_synthetic = pd.concat([df_synthetic.drop('Metrics', axis=1), pd.DataFrame(df_synthetic['Metrics'].tolist())], axis=1)
df_synthetic

,Model,Algorithm,RMSE,DEE,VAL,UGF,CVS
0,PQ,unfair,0.684380,0.012817,0.185479,0.000258,0.000192
1,PQ,EqualExp,0.674616,0.012017,0.200847,0.000442,0.000192
2,PQ,VAL,0.679315,0.012683,0.186601,0.000192,0.000425
3,PQ,UGF,0.683220,0.012467,0.178456,0.000167,0.000550
4,PQ,CVS,0.693884,0.012717,0.174996,0.000042,0.000325
5,AE,unfair,0.785640,0.004700,0.232037,0.001633,0.001367
6,AE,EqualExp,0.783555,0.007683,0.228033,0.001758,0.000875
7,AE,VAL,0.786629,0.006083,0.231353,0.001375,0.000308
8,AE,UGF,0.770288,0.017950,0.235738,0.003308,0.004042
9,AE,CVS,0.770027,0.018017,0.229629,0.001658,0.002142


---

In [ ]:
!pip install -q watermark
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-12-11 16:39:24

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

IPython   : 5.5.0
sys       : 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
matplotlib: 3.2.2
torch     : 1.10.0+cu111
pandas    : 1.1.5
numpy     : 1.19.5



---

**END**